In [2]:
import time
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np
from datetime import date
from lxml import html
#import pyautogui
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC


class MAQUINA():    
    def __init__(self, busca):

        self.busca = busca
        self.resultsDict = {\
            'nome' : [],
            'nEstrelas' : [],
            'nVotos' : [],
            'faixa de preço' : [],
            'endereço' : [],
            'site' : [],
            'fone' : [],
            'additional_info': []
        }
        self.resultsDict2 = {\
            'nome' : [],
            'data' : [],
            'eval' : []
        }
        self.wd = None
        self.iteration = 3
        self.size= 5
    def openBrowser(self):
        """
        abre o google, pesquisa pelo termo em 'busca' e clica em maps
        """

        options = webdriver.ChromeOptions()
        # INICIALIZAÇÃO DO DRIVER SELENIUM E INICIO DA BUSCA
        self.wd = webdriver.Edge(options=options)
        self.wd.get('https://www.google.com/')
        time.sleep(3)
        self.wd.maximize_window()
        time.sleep(1)
        self.wd.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(self.busca)#.send_keys(Keys.RETURN)
        time.sleep(1)
        self.wd.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(Keys.RETURN)
        time.sleep(2)
        self.wd.find_element(By.PARTIAL_LINK_TEXT, 'Maps').click()
        time.sleep(2)
    def makeSoup(self):
        """
        constrói a sopa do beautifulsoup
        """
        soup = BeautifulSoup(self.wd.page_source, 'html.parser')
    def scrollDown(self):
        #scroll down na pagina
        for i in range(3):
            elemento_scroll = self.wd.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
            elemento_scroll.send_keys(Keys.END)
            WebDriverWait(self.wd, 20).until(
                lambda x: x.execute_script("return document.readyState") == "complete")
            time.sleep(2)
            #print('esperou')
    def crawl(self):

        #EXECUTA A EXTRAÇÃO DOS DADOS CLICANDO NOS ITENS DA LISTA 1 A 1
        for element in range(self.size, 3, -2):

            try:

                # Localizar o elem}ento pai que possui o scroll
                elemento_pai = self.wd.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
                # Localizar o subelemento para o qual você deseja rolar
                subelemento = elemento_pai.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{element}]')
                # Rolar até o subelemento usando JavaScript
                self.wd.execute_script("arguments[0].scrollTop = arguments[1].offsetTop;", elemento_pai, subelemento)
                time.sleep(1)
                #abre o menu suspenso do local
                self.wd.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{element}]/div').click()
                #atualiza a sopa de letrinhas
                #espera até carregar aquela bosta
                WebDriverWait(self.wd, 20).until(
                lambda x: x.execute_script("return document.readyState") == "complete")
                time.sleep(4)
                soup = BeautifulSoup(self.wd.page_source, 'html.parser')
                # Converter o objeto BeautifulSoup para um objeto lxml
                root = html.fromstring(str(soup))
                #população do dict
                try:
                    self.resultsDict2['nome'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1/text()')[0])
                except:
                    self.resultsDict2['nome'].append('não encontrado')
                try:
                    self.resultsDict2['data'].append(self.wd.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[7]').text.split('\n'))
                except:
                    self.resultsDict2['data'].append('não encontrado')
                try:
                    self.resultsDict2['eval'].append(self.wd.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[23]/div/div[2]').text.replace(',', '.').split('\n'))
                except:
                    self.resultsDict2['eval'].append('não encontrado')

            #se nao carregar o próximo, tenta carregar mais a pagina
            except: self.iteration = element

        return pd.DataFrame(self.resultsDict2)
    def crawl2(self):

        #EXECUTA A EXTRAÇÃO DOS DADOS CLICANDO NOS ITENS DA LISTA 1 A 1
        for element in range(self.size, 3, -2):

            try:
                # Localizar o elemento pai que possui o scroll
                elemento_pai = self.wd.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
                # Localizar o subelemento para o qual você deseja rolar
                subelemento = elemento_pai.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{element}]')
                # Rolar até o subelemento usando JavaScript
                self.wd.execute_script("arguments[0].scrollTop = arguments[1].offsetTop;", elemento_pai, subelemento)
                time.sleep(0.5)

                #abre o menu suspenso do local
                self.wd.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{element}]').click()
                #atualiza a sopa de letrinhas
                #espera até carregar aquela bosta
                WebDriverWait(self.wd, 20).until(
                lambda x: x.execute_script("return document.readyState") == "complete")
                time.sleep(2)              
                soup1 = BeautifulSoup(self.wd.page_source, 'html.parser')
                soup = BeautifulSoup(self.wd.page_source, 'html.parser')
                # Converter o objeto BeautifulSoup para um objeto lxml
                root = html.fromstring(str(soup))
                #população do dict
                try:
                    self.resultsDict['nome'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1/text()')[0])
                except:
                    self.resultsDict['nome'].append('não encontrado')
                try:
                    self.resultsDict['nEstrelas'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]/text()')[0])
                except:
                    self.resultsDict['nEstrelas'].append('não encontrado')
                try:
                    self.resultsDict['nVotos'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span/text()')[0])
                except:
                    self.resultsDict['nVotos'].append('não encontrado')
                try:
                    self.resultsDict['faixa de preço'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/span/span/span/span[2]/span/span/text()')[0])
                except:
                    self.resultsDict['faixa de preço'].append('não encontrado')
                try:
                    self.resultsDict['endereço'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]/text()')[0])
                except:
                    self.resultsDict['endereço'].append('não encontrado')
                try:
                    self.resultsDict['site'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[5]/a/div/div[2]/div[1]/text()')[0])
                except:
                    self.resultsDict['site'].append('não encontrado')
                try:
                    self.resultsDict['fone'].append(root.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[6]/button/div/div[2]/div[1]/text()')[0])
                except:
                    self.resultsDict['fone'].append('não encontrado')
                try: 
                    self.resultsDict['additional_info'].append(self.wd.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]').text.split('\n'))
                except: 
                    self.resultsDict['additional_info'].append('não encontrado')

            #se nao carregar o próximo, tenta carregar mais a pagina
            except: self.iteration = element

        pd.DataFrame(self.resultsDict).to_csv(f'{self.busca}.csv')
        return pd.DataFrame(self.resultsDict)
    def sizeCalc(self):
        while True:
        #for i in range(1):
            soup = BeautifulSoup(self.wd.page_source, 'html.parser')
            self.size = len(soup.findAll('div', {'class' : 'TFQHme'}))
            self.scrollDown()
            time.sleep(1)
            soup = BeautifulSoup(self.wd.page_source, 'html.parser')
            size2 = len(soup.findAll('div', {'class' : 'TFQHme'}))
            if size2 > self.size:
                pass
            else: 
                self.size = 2 * self.size + 1
                break
            #o tamanho da lista real é 2* o tamanho dela encontrado em size + 1 
            #self.size = 2 * self.size + 1
            time.sleep(1)            

In [3]:
dfs = []
regioes = ['regiao norte']
busca = 'museus campinas  '
for r in regioes:
    busca = busca + r
    df = MAQUINA(busca)
    df.openBrowser()
    df.sizeCalc()
    df.crawl()
    dfs.append(pd.DataFrame(df.resultsDict2))
    df = pd.concat(dfs)

In [6]:
import re

def extrair_numeros_telefone(lista_strings):#funciona para lista de strings
    numeros_telefone = []
    padrao_telefone = r'\(?\d{2}\)?\s?\d{4,5}-?\d{4}'
    for string in lista_strings:
        numeros = re.findall(padrao_telefone, string)
        numeros_telefone.extend(numeros)
    return numeros_telefone

df['Telefones'] = df['data'].apply(extrair_numeros_telefone)
def extrair_numero(telefones):
    if len(telefones) > 0:
        return telefones[0]
    else:
        return None
# Aplicar a função à coluna 'Telefones' e criar uma nova coluna 'Telefones1'
df['Telefones1'] = df['Telefones'].apply(extrair_numero)

def filtrar_telefones(telefone):
    # captura os telefones iniciados com 199
    if str(telefone).startswith('(19) 9'):
        return telefone
    else:
        return np.nan
# Aplicar a função à coluna 'Telefones1' e criar a coluna 'Telefones2'
df['Telefones2'] = df['Telefones1'].apply(filtrar_telefones)


def remover_nao_numericos(texto):
    #deixa os telefones da coluna Telefones3 como uma string continua, ex: 1999123123. Para ser usada como input em outros lugares
    if isinstance(texto, str):
        return re.sub(r'\D', '', texto)
    else:
        return np.nan
df['Telefones3'] = df['Telefones2'].apply(remover_nao_numericos)
df

,nome,data,eval,Telefones,Telefones1,Telefones2,Telefones3
0,Museu de blindados,"[, Jardim Chapadão, Campinas - SP, , Fechado...",não encontrado,[],None,NaN,NaN
1,Casa Dos Animais Interessantes,"[, Bosque, Campinas - SP, Localizado em: Parq...",não encontrado,[(19) 3295-5850],(19) 3295-5850,NaN,NaN
2,Museu Exploratório de Ciências,"[, Av. Alan Turing, 1500 - Cidade Universitár...",não encontrado,[],None,NaN,NaN
3,Museu Carlos Gomes,"[, R. Bernardino de Campos, 989 - Centro, Cam...",não encontrado,[(19) 3231-2567],(19) 3231-2567,NaN,NaN
4,Museu da Cidade - Casa de Vidro,"[, Av. Dr. Heitor Penteado, 2145 - Parque Taq...",não encontrado,[],None,NaN,NaN
5,MACC - Museu de Arte Contemporânea de Campinas,[],não encontrado,[],None,NaN,NaN
6,Museu Universitário da PUC-Campinas,"[, Prédio da Antiga, Pastoral 1 - Avenida Dr....",não encontrado,[(19) 3343-5855],(19) 3343-5855,NaN,NaN
7,Museu do Forte Anhanguera,"[, Av. Sd. Passarinho, 1601 - Jardim Chapadão...",não encontrado,[(19) 99939-5963],(19) 99939-5963,(19) 99939-5963,19999395963
8,Museu do Negro de Campinas,[],não encontrado,[],None,NaN,NaN
9,Museu da Imagem e do Som de Campinas,[],não encontrado,[],None,NaN,NaN
